In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd

Fixed Model Parameters

In [2]:
alpha_1 = 3e-4
alpha_2 = 1.2
fk_max = 5
R_k = 0.5
v_1 = 1.2e2
v_2 = 14.4 *1e-1
#v_1_min = 0.5*v_1
#v_1_max = 1.5*v_1
#R_a = 0.3
 

Kill Function and Dimensionless ODE Model

In [3]:
def kill_function(R):
     f = fk_max*(R/(R + R_k))

     return f

def model(t, z, gamma, mu):

    dAdt = -alpha_1*(1 - z[1])*z[0]*z[2] + alpha_2*gamma*z[1]*z[2]
    dRdt = (alpha_1/gamma)*(1 - z[1])*z[0] - alpha_2*z[1]
    dSdt = -kill_function(z[1])*z[3]
    dCdt = v_1*((mu/20) - z[3])*(z[2] - z[3]) - v_2*z[3] - kill_function(z[1])*z[3]
    #dCdt = (v_1_min + (((z[1]**2)*(v_1_max - v_1_min))/(z[1]**2 + R_a**2)))*((mu/20) - z[3])*(z[2] - z[3]) - v_2*z[3] - kill_function(z[1])*z[3]
    dzdt = [dAdt, dRdt, dSdt, dCdt]

    return dzdt

ICs and Simulation Parameters

In [4]:
z0 = [1, 0, 1, 0] 
t = np.arange(0, 1.1, 0.1)
t_span = [0, 1]
mu_vals = [1.25, 2.5, 5, 10, 20]
gamma_vals = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
noise_vals = [0.01, 0.05, 0.1, 0.2]

Create Dataframe

In [5]:
df = pd.DataFrame(columns=['t', 'mu', 'gamma', 'noise level', 'A', 'R', 'S', 'C'])


In [6]:

for gamma in gamma_vals:
    for mu in mu_vals:
        for noise_level in noise_vals:
            j=0
            while j<10:
                
                z = solve_ivp(model, t_span, z0, args = (gamma, mu), method ='Radau', t_eval = t)

                # add noise
                A_array = z.y[0]
                R_array = z.y[1]
                S_array = z.y[2]
                C_array = z.y[3]

                for i in range(len(A_array)):
                    A_array[i] = A_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                    R_array[i] = R_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                    S_array[i] = S_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                    C_array[i] = C_array[i]*(1 + np.random.uniform(-noise_level, noise_level))

                # add data to dataframe
                for i in range(len(A_array)):
                    df = df.append({'t' : t[i],
                                    'mu' : mu,
                                    'gamma' : gamma,
                                    'noise level' : noise_level,
                                    'A' : A_array[i],
                                    'R' : R_array[i],
                                    'S' : S_array[i],
                                    'C' : C_array[i]}, ignore_index=True)
                
                j += 1

        

In [7]:
print(df)

         t     mu     gamma  noise level         A         R         S  \
0      0.0   1.25  1.000000         0.01  1.008632  0.000000  1.003011   
1      0.1   1.25  1.000000         0.01  1.000554  0.000028  1.008602   
2      0.2   1.25  1.000000         0.01  0.995045  0.000053  1.008071   
3      0.3   1.25  1.000000         0.01  0.991256  0.000075  1.008931   
4      0.4   1.25  1.000000         0.01  1.002301  0.000095  0.997255   
...    ...    ...       ...          ...       ...       ...       ...   
15395  0.6  20.00  0.000001         0.20  1.192880  0.991283  0.142314   
15396  0.7  20.00  0.000001         0.20  0.804617  1.133061  0.115207   
15397  0.8  20.00  0.000001         0.20  1.143772  0.954852  0.094166   
15398  0.9  20.00  0.000001         0.20  1.052254  0.878521  0.061302   
15399  1.0  20.00  0.000001         0.20  1.140456  0.913857  0.047554   

              C  
0      0.000000  
1      0.062041  
2      0.061485  
3      0.062138  
4      0.061272  
...

Write Dataframe to CSV

In [8]:
df.to_csv('/home/sabsr3/Rotations/ADCC/ADCC-Project/Data/basic_model_synthetic_data_Hill_v1.csv')